Detección de caras con webcam

In [1]:
import cv2
import time
import FaceNormalizationUtils as faceutils
# My face detectors interface
import FaceDetectors

# Filtro

In [ ]:
## Codigo Louka

# JUEGO 'FLY-VISION'

In [14]:
import cv2
import numpy as np
import time
import random
import FaceNormalizationUtils as faceutils
import FaceDetectors

normalizatorHS = faceutils.Normalization()
FDet = FaceDetectors.FaceDetector()

font = cv2.FONT_HERSHEY_SIMPLEX
cap = cv2.VideoCapture(1)
if not cap.isOpened():
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        cap = cv2.VideoCapture(0)
        if not cap is not None:
            print('Camera error')
            exit(0)
        else:
            print('Camera 0')
    else:
        print('Camera 1')
else:
    print('Camera 0')

cap.set(3, 640)
cap.set(4, 480)

imodoF = 0
imodoE = 0
score = 0
speed_increment_counter = 0
obstacle_speed = 5
obstacle_speed_increment = 1  # Aumento de velocidad cada 2 puntos
max_speed = 20  # Velocidad máxima de los obstáculos

bird_y = 240
bird_velocity = 0
gravity = 1
game_over = False
game_started = False
space_pressed = False
bird_passed_obstacle = False  # Variable para controlar el puntaje

# Crear un fondo azul sólido
blue_bg = np.zeros((480, 640, 3), dtype=np.uint8)
blue_bg[:, :] = (255, 0, 0)  # Color azul

# Cargar la imagen del avión
avion = cv2.imread('./images/AVION-removebg-preview.png', cv2.IMREAD_UNCHANGED)
# Asegurarse de que la imagen del avión tenga 4 canales (RGBA)
if avion.shape[2] == 4:
    # Crear una máscara para los píxeles transparentes
    mask = avion[:, :, 3]

    # Convertir la imagen del avión a un formato de 3 canales (BGR) y rellenar los píxeles transparentes con un color de fondo
    background_color = (255, 0, 0)  # Color de fondo deseado (blanco en este ejemplo)
    avion = cv2.cvtColor(avion, cv2.COLOR_BGRA2BGR)
    avion[mask != 255] = background_color

avion = cv2.resize(avion, (70, 60))
obstacles = []

obstacle_width = 50  # Ancho de los obstáculos

def reset_game():
    global bird_y, bird_velocity, score, game_over, game_started, obstacles, speed_increment_counter, obstacle_speed
    bird_y = 240
    bird_velocity = 0
    score = 0
    game_over = False
    game_started = False
    obstacles = []
    speed_increment_counter = 0
    obstacle_speed = 5

def generate_obstacle():
    x = 640
    upper_height = random.randint(50, 250)
    lower_height = 350 - upper_height
    return (x, upper_height, lower_height)

reset_game()

while True:
    t = time.time()
    ret, frame = cap.read()
    B, G, R = cv2.split(frame)

    values = FDet.SingleFaceEyesDetection(frame, FDet.FaceDetectors[imodoF], FDet.EyeDetectors[imodoE])
    if values is not None:
        face, eyes, shape = values
        [x, y, w, h] = face
        if x > -1:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            [lex, ley, rex, rey] = eyes
            if lex > -1:
                for (x, y) in shape:
                    cv2.circle(frame, (x, y), 2, (255, 255, 255), -1)
                cv2.circle(frame, ((int)(lex), (int)(ley)), 4, (0, 0, 255), -1)
                cv2.circle(frame, ((int)(rex), (int)(rey)), 4, (0, 255, 0), -1)
                normalizatorHS.normalize_gray_img(B, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Bnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(G, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Gnorm = normalizatorHS.normf_image
                normalizatorHS.normalize_gray_img(R, lex, ley, rex, rey, faceutils.Kind_wraping.HS)
                Rnorm = normalizatorHS.normf_image
                NormBGR = cv2.merge((Bnorm, Gnorm, Rnorm))

    # Combinar el fondo azul con la imagen de la c
    # Combinar el fondo azul con la imagen de la claridad
    frame = blue_bg.copy()

    if not game_over:
        if game_started:
            bird_y = ley

            # Mover y actualizar los obstáculos existentes
            new_obstacles = []
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                x -= obstacle_speed
                if x >= -obstacle_width:
                    new_obstacles.append((x, upper_height, lower_height))
            obstacles = new_obstacles

            # Generar nuevos obstáculos
            if len(obstacles) == 0 or obstacles[-1][0] < 400:  # Generar un nuevo obstáculo cuando el último esté cerca del centro
                obstacles.append(generate_obstacle())

            # Detección de colisión con los obstáculos
            bird_x = 100
            bird_radius = 20
            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                if (
                    bird_x + bird_radius > x
                    and bird_x - bird_radius < x + obstacle_width
                    and (bird_y - bird_radius < upper_height or bird_y + bird_radius > 480 - lower_height)
                ):
                    game_over = True

            # Incrementar puntos cuando el obstáculo haya pasado completamente
            if obstacles and obstacles[0][0] + obstacle_width < bird_x - bird_radius:
                if not bird_passed_obstacle:
                    score += 1
                    bird_passed_obstacle = True
            else:
                bird_passed_obstacle = False

            # Dibujar el avión en lugar de un círculo verde
            frame[ley:ley+avion.shape[0], 100:100+avion.shape[1]] = avion

            for obstacle in obstacles:
                x, upper_height, lower_height = obstacle
                cv2.rectangle(frame, (x, 0), (x + obstacle_width, upper_height), (0, 0, 255), -1)
                cv2.rectangle(frame, (x, 480 - lower_height), (x + obstacle_width, 480), (0, 0, 255), -1)

            # Incrementar velocidad cada 2 puntos
            if score > 0 and score % 2 == 0 and obstacle_speed < max_speed:
                obstacle_speed += obstacle_speed_increment

        if not game_started:
            cv2.putText(frame, "Press 'Space' to Start", (200, 200), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    else:
        cv2.putText(frame, "Game Over", (280, 200), font, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.putText(frame, "Score: " + str(score), (10, 20), font, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Cam', frame)

    key = cv2.waitKey(40)
    if key == 27:  # Tecla Esc para salir
        break
    elif key == 32:  # Tecla Espacio
        if not game_started:
            if not space_pressed:
                reset_game()
                game_started = True
                space_pressed = True
        else:
            space_pressed = False

cap.release()
cv2.destroyAllWindows()


Camera 0
